In [2]:
%%bash
mkdir -p ./ex_toy02

In [3]:
%%bash
cat > ./ex_toy02/normal_mixture.stan << EOF
data {
    int<lower=1> N; // number of data points; lower = 1 means to constrain N > 0
    real X[N];      // X[i] = gene expression for individual i
}

parameters {
    real <lower=0, upper=1> r; // mixture proportion
    real mu;                   // mu = mean of individuals w/ decreased expression
    real<lower=0.001> sigma;   // sigma = variance of expression > 0
}

model {
    r ~ beta(2,2);
    mu ~ normal(0,10);
    sigma ~ inv_gamma(1,1);
    for (i in 1:N) 
        target += log_sum_exp(
            log(r)   + normal_lpdf(X[i] | mu, sigma),
            log(1-r) + normal_lpdf(X[i] |  0, sigma));
}
EOF

In [4]:
%%bash
### load modules
module load gcc
module load tbb

### set directory
STAN_PATH=/data/reddylab/Kuei/exe/cmdstan
FD_WRK=$(pwd)
echo $FD_WRK

### compile model
make -C $STAN_PATH ${FD_WRK}/ex_toy02/normal_mixture

/gpfs/fs1/data/reddylab/Kuei/learn/learn_stan
make: Entering directory `/gpfs/fs1/data/reddylab/Kuei/exe/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  --o=/gpfs/fs1/data/reddylab/Kuei/learn/learn_stan/ex_toy02/normal_mixture.hpp /gpfs/fs1/data/reddylab/Kuei/learn/learn_stan/ex_toy02/normal_mixture.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include   -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_5.7.0/include    -DBOOST_DISABLE_ASSERTS         -c -Wno-ignored-attributes   -x c++ -o /gpfs/fs1/data/reddylab/Kuei/learn/learn_stan/ex_toy02/normal_mixture.o /gpfs/fs1/data/reddylab/Kuei/learn/learn_stan/ex_toy02/normal_mixture.hpp
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attrib

## Generate data

In [6]:
import numpy as np
import json

In [7]:
### simulation
np.random.seed(123)
pos = np.random.normal(loc= 0, scale=0.3, size=50)
neg = np.random.normal(loc=-1, scale=0.3, size=50)

print("pos")
print("Mean:", np.around(pos.mean(), 2))
print("SD:  ", np.around(pos.std(),  2))

print("neg")
print("Mean:", np.around(neg.mean(), 2))
print("SD:  ", np.around(neg.std(),  2))

pos
Mean: 0.0
SD:   0.36
neg
Mean: -0.99
SD:   0.32


In [8]:
### generate data file
dct = dict()
dct["N"] = 100
dct["X"] = pos.tolist() + neg.tolist()

### store the data
fpath = "./ex_toy02/inputs.json"
with open(fpath, "w") as file:
    json.dump(dct, file)

## Running the model

In [9]:
%%bash
module load gcc
./ex_toy02/normal_mixture sample \
    thin=1 num_samples=1000 num_warmup=50 \
    data file=./ex_toy02/inputs.json \
    output file=./ex_toy02/output.csv

method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 50
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delta = 0.80000000000000004 (Default)
      kappa = 0.75 (Default)
      t0 = 10 (Default)
      init_buffer = 75 (Default)
      term_buffer = 50 (Default)
      window = 25 (Default)
    algorithm = hmc (Default)
      hmc
        engine = nuts (Default)
          nuts
            max_depth = 10 (Default)
        metric = diag_e (Default)
        metric_file =  (Default)
        stepsize = 1 (Default)
        stepsize_jitter = 0 (Default)
id = 0 (Default)
data
  file = ./ex_toy02/inputs.json
init = 2 (Default)
random
  seed = 3001794802 (Default)
output
  file = ./ex_toy02/output.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)


Gradient evaluation took 0.000233 seconds
1000 transi

In [13]:
%%bash
module load gcc
STAN_PATH=/data/reddylab/Kuei/exe/cmdstan
$STAN_PATH/bin/stansummary ./ex_toy02/output.csv

Inference for Stan model: normal_mixture_model
1 chains: each with iter=(1000); warmup=(0); thin=(1); 1000 iterations saved.

Warmup took 0.038 seconds
Sampling took 0.46 seconds

                 Mean     MCSE   StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat

lp__              -93  8.6e-02      1.4    -96    -92    -91      278      603      1.0
accept_stat__    0.91  3.2e-03  1.1e-01   0.67   0.95    1.0  1.2e+03  2.7e+03  1.0e+00
stepsize__      0.062      nan  2.7e-16  0.062  0.062  0.062      nan      nan      nan
treedepth__       2.6  2.4e-02  6.8e-01    2.0    3.0    4.0  8.0e+02  1.7e+03  1.0e+00
n_leapfrog__      8.0  1.5e-01  4.7e+00    3.0    7.0     15  9.8e+02  2.1e+03  1.0e+00
divergent__      0.00      nan  0.0e+00   0.00   0.00   0.00      nan      nan      nan
energy__           94  1.2e-01  2.0e+00     92     94     98  3.0e+02  6.4e+02  1.0e+00

r                0.53  5.4e-03    0.077   0.41   0.53   0.65      202      437      1.0
mu              -0.93  5.2